In [5]:
!pip install Flask
!pip install flask-cors
!pip install pyodbc


In [6]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pyodbc

In [17]:
from flask import Flask, jsonify, request, render_template
from flask_cors import CORS
import os
import pyodbc
template_dir = r'D:\Projects\Year2-Term1\Project Data Base\templates'
app = Flask(__name__, template_folder=template_dir)

CORS(app)
server = os.getenv('DB_SERVER')
database = os.getenv('DB_NAME')
username = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
driver = os.getenv('DB_DRIVER')

conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')

cursor = conn.cursor() 
#-------------------------------------------------------------------------------------------------------------------------
# Routes for Guests
@app.route('/')
def home():
    return render_template("Frontend.html")

@app.route("/Guests", methods=["GET"])
def view_Guests():
    cursor.execute("select * from Guests")
    Gus = cursor.fetchall()
    Guests = []   
    for guest in Gus:  
        Guests.append({
            "guest_id": guest[0], 
            "guest_fname": guest[1],
            "guest_lname": guest[2],
            "guest_ssn": guest[3],
            "guest_email": guest[4],
            "guest_phone": guest[5]
        })
    return jsonify(Guests)

@app.route("/Guests", methods=["POST"])
def add_Guest():
    Guests = request.json
    ssn = Guests["guest_ssn"]
    phone = Guests["guest_phone"]
    if len(ssn) != 14 or not ssn.isdigit():
        return jsonify({"error": "SSN must be 14 digits"}), 400

    if len(phone) != 11 or not phone.isdigit():
        return jsonify({"error": "Phone must be 11 digits"}), 400
    try:
        cursor.execute('''insert into Guests (guest_fname, guest_lname, guest_ssn, guest_email, guest_phone) values (? ,? ,? ,? ,?)''', 
                    (Guests["guest_fname"], Guests["guest_lname"], Guests["guest_ssn"], Guests["guest_email"], Guests["guest_phone"]))    
        conn.commit()  
    except pyodbc.IntegrityError:
        return jsonify({"error": "SSN already exists"}), 400    
    return jsonify({"message": "Guests added successfully!"})

@app.route('/Guests/<int:guest_id>', methods=['PUT'])
def update_Guest(guest_id):
    data = request.get_json()
    fname = data.get("guest_fname")
    lname = data.get("guest_lname")
    ssn = data.get("guest_ssn")
    email = data.get("guest_email")
    phone = data.get("guest_phone")

    if not ssn.isdigit() or len(ssn) != 14:
        return jsonify({"error": "SSN must be 14 digits"}), 400

    if not phone.isdigit() or len(phone) != 11:
        return jsonify({"error": "Phone must be 11 digits"}), 400

    cursor.execute("""
        UPDATE Guests
        SET guest_fname = ?, guest_lname = ?, guest_ssn = ?, guest_email = ?, guest_phone = ?
        WHERE guest_id = ?
    """, (fname, lname, ssn, email, phone, guest_id))
    conn.commit()

    return jsonify({"message": "Guest updated successfully"})

@app.route('/Guests/<int:guest_id>', methods=['DELETE'])
def delete_Guest(guest_id):
    try:
        cursor.execute("SELECT * FROM Booking WHERE guest_id = ?", (guest_id,))
        booking = cursor.fetchone()

        if booking:
            return jsonify({"error": "Cannot delete guest. They have existing bookings."}), 400

        cursor.execute("DELETE FROM Guests WHERE guest_id = ?", (guest_id,))
        conn.commit()
        return jsonify({"message": "Guest deleted successfully"}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500
    

#-------------------------------------------------------------------------------------------------------------------------
# Routes for Employees
@app.route("/Employees", methods=["GET"])
def view_Employees():
    cursor.execute("select * from Employees")
    Emps = cursor.fetchall()
    Employees = []   
    for Employee in Emps:  
        Employees.append({
            "employee_id": Employee[0], 
            "manager_id": Employee[1],
            "employee_lname": Employee[2],
            "employee_fname": Employee[3],
            "employee_ssn": Employee[4],
            "employee_role": Employee[5],
            "employee_phone": Employee[6]
        })
    return jsonify(Employees)


@app.route("/Employees", methods=["POST"])
def add_Employees():
    Employees = request.json
    ssn = Employees.get("employee_ssn")

   
    if len(ssn) != 14 or not ssn.isdigit():
        return jsonify({"error": "employee_ssn must be exactly 14 digits"}), 400

  
    cursor.execute("SELECT * FROM Employees WHERE employee_ssn = ?", (ssn,))
    if cursor.fetchone() is not None:
        return jsonify({"error": "employee_ssn already exists"}), 400

    try:
        cursor.execute('''
            insert into Employees 
            (manager_id, employee_lname, employee_fname, employee_ssn, employee_role , employee_phone) 
            values (?, ?, ?, ?, ?, ?)
        ''', (
            Employees.get("manager_id"),
            Employees.get("employee_lname"),
            Employees.get("employee_fname"),
            ssn,
            Employees.get("employee_role"),
            Employees.get("employee_phone")
        ))    
        conn.commit()   
        return jsonify({"message": "Employee added successfully!"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route('/Employees/<int:employee_id>', methods=['PUT'])
def update_Employee(employee_id):
    Employees = request.get_json()
    new_manager_id = Employees.get("manager_id")
    new_employee_lname = Employees.get("employee_lname")
    new_employee_fname = Employees.get("employee_fname")
    new_employee_ssn = Employees.get("employee_ssn")
    new_employee_role = Employees.get("employee_role")
    new_employee_phone = Employees.get("employee_phone")

    if new_employee_ssn is None or len(new_employee_ssn) != 14 or not new_employee_ssn.isdigit():
        return jsonify({"error": "employee_ssn must be exactly 14 digits"}), 400

    try:
        cursor.execute("""
            update Employees
            set manager_id = ?, employee_lname = ?, employee_fname = ?, employee_ssn = ?, employee_role = ? , employee_phone = ?
            where employee_id = ?
        """, (new_manager_id, new_employee_lname, new_employee_fname, new_employee_ssn, new_employee_role, new_employee_phone, employee_id))
        conn.commit()
        return jsonify({"message": "Employees updated successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route('/Employees/<int:employee_id>', methods=['DELETE'])
def delete_Employee(employee_id):
    cursor.execute("delete from Employees where employee_id = ?", (employee_id,))
    conn.commit()
    return jsonify({"message": "Employees deleted successfully"})


#-------------------------------------------------------------------------------------------------------------------------
# Routes for Rooms
@app.route("/Rooms", methods=["GET"])
def view_Rooms():
    cursor.execute("select * from Rooms")
    roos = cursor.fetchall()
    Rooms = []   
    for Room in roos:  
        Rooms.append({
            "room_id": Room[0],
            "room_type_id": Room[1],
            "room_floor": Room[2],
            "room_status": Room[3]
        })
    return jsonify(Rooms)

@app.route("/Rooms", methods=["POST"])
def add_Rooms():
    data = request.get_json()

    cursor.execute("SELECT ISNULL(MAX(room_id), 0) + 1 FROM Rooms")
    new_room_id = cursor.fetchone()[0]

    cursor.execute('''
        INSERT INTO Rooms (room_id, room_type_id, room_floor, room_status) 
        VALUES (?, ?, ?, ?)
    ''', (
        new_room_id,
        data["room_type_id"], 
        data["room_floor"], 
        data["room_status"]
    ))
    
    conn.commit()
    return jsonify({"message": "Room added successfully!", "room_id": new_room_id}), 201

@app.route('/Rooms/<int:room_id>', methods=['PUT'])
def update_Rooms(room_id):
    Rooms = request.get_json()
    new_room_type_id = Rooms.get("room_type_id")
    new_room_floor = Rooms.get("room_floor")
    new_room_status = Rooms.get("room_status")

    cursor.execute("""
        update Rooms
        set room_type_id = ?, room_floor = ?, room_status = ?
        where room_id = ?""",
        (new_room_type_id, new_room_floor, new_room_status,room_id))
    conn.commit()
    return jsonify({"message": "Room updated successfully"}), 200

@app.route('/Rooms/<int:room_id>', methods=['DELETE'])
def delete_Rooms(room_id):
    cursor.execute("delete from Rooms where room_id = ?", (room_id,))
    conn.commit()
    return jsonify({"message": "Rooms deleted successfully"})

#-------------------------------------------------------------------------------------------------------------------------
# Routes for RoomTypes
@app.route("/Room_type", methods=["GET"])
def view_Room_type():
    cursor.execute("SELECT * FROM Room_type ORDER BY room_type_id")
    roos = cursor.fetchall()
    Room_type = []   
    for Room_t in roos:  
        Room_type.append({
            "room_type_id": Room_t[0],
            "type_price": float(Room_t[1]),
            "capacity": Room_t[2],
            "room_type": Room_t[3]
        })
    return jsonify(Room_type)


@app.route("/Room_type", methods=["POST"])
def add_Room_type():
    data = request.json
    
    if not data.get("room_type") or not data.get("type_price") or not data.get("capacity"):
        return jsonify({"error": "All fields are required"}), 400
    
    if float(data["type_price"]) <= 0:
        return jsonify({"error": "Price must be greater than 0"}), 400
    
    if int(data["capacity"]) <= 0:
        return jsonify({"error": "Capacity must be at least 1"}), 400

    try:
        cursor.execute("SELECT ISNULL(MAX(room_type_id), 0) + 1 FROM Room_type")
        new_id = cursor.fetchone()[0]
        
        cursor.execute('''
            INSERT INTO Room_type (room_type_id, type_price, capacity, room_type) 
            VALUES (?, ?, ?, ?)
        ''', (new_id, data["type_price"], data["capacity"], data["room_type"]))
        
        conn.commit()   
        return jsonify({
            "message": "Room type added successfully!", 
            "room_type_id": new_id
        }), 201
        
    except pyodbc.IntegrityError as e:
        return jsonify({"error": "Room type already exists or invalid data"}), 400
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route('/Room_type/<int:room_type_id>', methods=['PUT'])
def update_Room_type(room_type_id):
    data = request.get_json()
    
    if not data.get("room_type") or not data.get("type_price") or not data.get("capacity"):
        return jsonify({"error": "All fields are required"}), 400
    
    if float(data["type_price"]) <= 0:
        return jsonify({"error": "Price must be greater than 0"}), 400
    
    if int(data["capacity"]) <= 0:
        return jsonify({"error": "Capacity must be at least 1"}), 400

    try:
    
        cursor.execute("SELECT room_type_id FROM Room_type WHERE room_type_id = ?", (room_type_id,))
        if not cursor.fetchone():
            return jsonify({"error": "Room type not found"}), 404
        
        cursor.execute("""
            UPDATE Room_type
            SET type_price = ?, capacity = ?, room_type = ?
            WHERE room_type_id = ?
        """, (data["type_price"], data["capacity"], data["room_type"], room_type_id))
        
        conn.commit()
        return jsonify({"message": "Room type updated successfully"}), 200
        
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route('/Room_type/<int:room_type_id>', methods=['DELETE'])
def delete_Room_type(room_type_id):
    try:

        cursor.execute("SELECT COUNT(*) FROM Rooms WHERE room_type_id = ?", (room_type_id,))
        count = cursor.fetchone()[0]
        
        if count > 0:
            return jsonify({
                "error": f"Cannot delete! This room type is used by {count} room(s)"
            }), 400
        
       
        cursor.execute("SELECT room_type_id FROM Room_type WHERE room_type_id = ?", (room_type_id,))
        if not cursor.fetchone():
            return jsonify({"error": "Room type not found"}), 404
        
        cursor.execute("DELETE FROM Room_type WHERE room_type_id = ?", (room_type_id,))
        conn.commit()
        return jsonify({"message": "Room type deleted successfully"}), 200
        
    except Exception as e:
        return jsonify({"error": str(e)}), 500

#-------------------------------------------------------------------------------------------------------------------------
# Routes for Booking
@app.route("/Booking", methods=["GET"])
def view_Booking():
    try:
        cursor.execute("SELECT * FROM Booking")
        Bookings = cursor.fetchall()
        Booking_list = []   
        for book in Bookings:  
            Booking_list.append({
                "booking_id": book[0],
                "guest_id": book[1],
                "room_id": book[2],
                "employee_id": book[3],
                "check_IN_date": book[4],
                "check_OUT_date": book[5],
                "Price": book[6],
                "booking_date": book[7],
                "room_state": book[8]
            })
        return jsonify(Booking_list), 200
    except Exception as e:
        return jsonify({"error": "Error fetching bookings: " + str(e)}), 500


@app.route("/Booking", methods=["POST"])
def add_Booking():
    try:
        Booking = request.json

      
        required_fields = ["guest_id","room_id","employee_id","check_IN_date","check_OUT_date","Price","room_state"]
        for field in required_fields:
            if field not in Booking or Booking[field] in [None, ""]:
                return jsonify({"error": f"{field} is required"}), 400

        if Booking["Price"] < 0:
            return jsonify({"error": "Price cannot be negative"}), 400

        if Booking["check_IN_date"] > Booking["check_OUT_date"]:
            return jsonify({"error": "Check-in date must be before check-out date"}), 400

        allowed_states = ["confirmed","pending","cancelled","completed"]
        if Booking["room_state"] not in allowed_states:
            return jsonify({"error": "Invalid room state"}), 400

      
        cursor.execute('''
            INSERT INTO Booking 
            (guest_id, room_id, employee_id , check_IN_date ,check_OUT_date, Price ,booking_date, room_state) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', 
            (Booking["guest_id"], Booking["room_id"], Booking["employee_id"],
             Booking["check_IN_date"], Booking["check_OUT_date"], Booking["Price"],
             Booking.get("booking_date", ""), Booking["room_state"]))    
        conn.commit()   
        return jsonify({"message": "Booking added successfully!"}), 201

    except Exception as e:
        return jsonify({"error": "Failed to add booking: " + str(e)}), 500



@app.route('/Booking/<int:booking_id>', methods=['PUT'])
def update_Booking(booking_id):
    try:
        Booking = request.get_json()

        required_fields = ["guest_id","room_id","employee_id","check_IN_date","check_OUT_date","Price","room_state"]
        for field in required_fields:
            if field not in Booking or Booking[field] in [None, ""]:
                return jsonify({"error": f"{field} is required"}), 400

        if Booking["Price"] < 0:
            return jsonify({"error": "Price cannot be negative"}), 400

        if Booking["check_IN_date"] > Booking["check_OUT_date"]:
            return jsonify({"error": "Check-in date must be before check-out date"}), 400

        allowed_states = ["confirmed","pending","cancelled","completed"]
        if Booking["room_state"] not in allowed_states:
            return jsonify({"error": "Invalid room state"}), 400

        cursor.execute("""
            UPDATE Booking
            SET guest_id=?, room_id=?, employee_id=?, check_IN_date=?, check_OUT_date=?, 
                Price=?, booking_date=?, room_state=?
            WHERE booking_id=?""",
            (Booking["guest_id"], Booking["room_id"], Booking["employee_id"], 
             Booking["check_IN_date"], Booking["check_OUT_date"], Booking["Price"],
             Booking.get("booking_date", ""), Booking["room_state"], booking_id))
        conn.commit()

        return jsonify({"message": "Booking updated successfully"}), 200

    except Exception as e:
        return jsonify({"error": "Failed to update booking: " + str(e)}), 500



@app.route('/Booking/<int:booking_id>', methods=['DELETE'])
def delete_Booking(booking_id):
    try:
        cursor.execute("DELETE FROM Booking WHERE booking_id = ?", (booking_id,))
        conn.commit()
        return jsonify({"message": "Booking deleted successfully"}), 200
    except Exception as e:
        
        if 'FK__' in str(e):
            return jsonify({"error": "Cannot delete this booking because it is linked to another table."}), 400
        return jsonify({"error": str(e)}), 500

#-------------------------------------------------------------------------------------------------------------------------
# Routes for Hotel_Service
@app.route("/hotel_Service", methods=["GET"])
def view_hotel_Service():
    try:
        cursor.execute("SELECT * FROM hotel_Service")
        services = cursor.fetchall()
        hotel_Services = []
        for service in services:
            hotel_Services.append({
                "service_id": service[0],
                "name_service": service[1],
                "price": service[2]
            })
        return jsonify(hotel_Services)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/hotel_Service", methods=["POST"])
def add_hotel_Service():
    try:
        hotel_Service = request.json
        cursor.execute(
            "INSERT INTO hotel_Service (service_id, name_service, price) VALUES (?, ?, ?)",
            (hotel_Service["service_id"], hotel_Service["name_service"], hotel_Service["price"])
        )
        conn.commit()
        return jsonify({"message": "hotel_Service added successfully!"}), 201
    except Exception as e:
        
        if "23000" in str(e):
            return jsonify({"error": "Service with this ID already exists!"}), 400
        return jsonify({"error": str(e)}), 500
@app.route('/hotel_Service/<int:service_id>', methods=['PUT'])
def update_hotel_Service(service_id):
    try:
        hotel_Service = request.get_json()
        new_name_service = hotel_Service.get("name_service")
        new_price = hotel_Service.get("price")
        cursor.execute(
            "UPDATE hotel_Service SET name_service = ?, price = ? WHERE service_id = ?",
            (new_name_service, new_price, service_id)
        )
        conn.commit()
        if cursor.rowcount == 0:
            return jsonify({"error": "Service not found"}), 404
        return jsonify({"message": "hotel_Service updated successfully"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/hotel_Service/<int:service_id>', methods=['DELETE'])
def delete_hotel_Service(service_id):
    try:
        cursor.execute("DELETE FROM hotel_Service WHERE service_id = ?", (service_id,))
        conn.commit()
        if cursor.rowcount == 0:
            return jsonify({"error": "Service with this ID does not exist!"}), 404
        return jsonify({"message": "hotel_Service deleted successfully"}), 200
    except Exception as e:
       
        if "REFERENCE" in str(e) or "foreign key" in str(e).lower():
            return jsonify({"error": "Cannot delete this service because it is linked to existing bookings!"}), 400

        return jsonify({"error": "Failed to delete service. Please try again or check the server."}), 500

#-------------------------------------------------------------------------------------------------------------------------
# Routes for Booking_service
@app.route("/Booking_service", methods=["GET"])
def view_Booking_service():
    cursor.execute("select * from Booking_service")
    bookings_services = cursor.fetchall()
    Booking_services = []   
    for booking_service in bookings_services:

        Booking_services.append({
            "booking_id": booking_service[0],
            "service_id": booking_service[1],
            "quantity": booking_service[2]
        })
    return jsonify(Booking_services)

@app.route("/Booking_service", methods=["POST"])
def add_Booking_service():
    try:
        Booking_service = request.json
        cursor.execute(
            "INSERT INTO Booking_service (booking_id, service_id, quantity) VALUES (?, ?, ?)",
            (Booking_service["booking_id"], Booking_service["service_id"], Booking_service["quantity"])
        )
        conn.commit()
        return jsonify({"message": "Booking_service added successfully!"}), 201
    except Exception as e:
        if "23000" in str(e):
            return jsonify({"error": "This booking already has this service!"}), 400
        return jsonify({"error": str(e)}), 500
@app.route('/Booking_service/<int:booking_id>/<int:service_id>', methods=['PUT'])
def update_Booking_service(booking_id, service_id):
    try:
        Booking_service = request.get_json()
        new_quantity = Booking_service.get("quantity")
        cursor.execute(
            "UPDATE Booking_service SET quantity = ? WHERE booking_id = ? AND service_id = ?",
            (new_quantity, booking_id, service_id)
        )
        conn.commit()
        if cursor.rowcount == 0:  
            return jsonify({"error": "Booking service not found"}), 404
        return jsonify({"message": "Booking_service updated successfully"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    
@app.route('/Booking_service/<int:booking_id>/<int:service_id>', methods=['DELETE'])
def delete_Booking_service(booking_id, service_id):
    try:
        cursor.execute(
            "DELETE FROM Booking_service WHERE booking_id = ? AND service_id = ?",
            (booking_id, service_id)
        )
        conn.commit()
        if cursor.rowcount == 0:  
            return jsonify({"error": "Booking service not found"}), 404
        return jsonify({"message": "Booking_service deleted successfully"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
#-------------------------------------------------------------------------------------------------------------------------
# GET all payments
@app.route("/Payment", methods=["GET"])
def view_Payment():
    cursor.execute("SELECT * FROM Payment")
    payments = cursor.fetchall()
    Payment = []   
    for payment in payments:
        Payment.append({
            "payment_id": payment[0],
            "booking_id": payment[1],
            "payment_type": payment[2],
            "amount": payment[3],
            "Payment_date": payment[4]
        })
    return jsonify(Payment)

@app.route("/Payment", methods=["POST"])
def add_Payment():
    Payment = request.json

    missing_fields = [f for f in ["payment_id", "booking_id", "payment_type", "amount", "Payment_date"] if not Payment.get(f)]
    if missing_fields:
        return jsonify({"error": f"The following fields are required: {', '.join(missing_fields)}"}), 400

    
    cursor.execute("SELECT * FROM Payment WHERE payment_id = ?", (Payment["payment_id"],))
    if cursor.fetchone():
        return jsonify({"error": f"Payment ID {Payment['payment_id']} already exists"}), 400

    try:
        cursor.execute('''INSERT INTO Payment (payment_id, booking_id, payment_type, amount, Payment_date)
                          VALUES (?, ?, ?, ?, ?)''',
                       (Payment["payment_id"], Payment["booking_id"], Payment["payment_type"], Payment["amount"], Payment["Payment_date"]))
        conn.commit()
        return jsonify({"message": "Payment added successfully!"}), 201
    except Exception as e:
        return jsonify({"error": "Database error: " + str(e)}), 500


@app.route('/Payment/<int:payment_id>/<int:booking_id>', methods=['PUT'])
def update_Payment(payment_id, booking_id):
    Payment = request.get_json()
    new_payment_type = Payment.get("payment_type")
    new_amount = Payment.get("amount")
    new_Payment_date = Payment.get("Payment_date")

    if not new_payment_type or not new_amount or not new_Payment_date:
        return jsonify({"error": "All fields are required"}), 400

    cursor.execute("""
        UPDATE Payment
        SET payment_type = ?, amount = ?, Payment_date = ?
        WHERE payment_id = ? AND booking_id = ?""",
        (new_payment_type, new_amount, new_Payment_date, payment_id, booking_id))
    conn.commit()

  
    cursor.execute("SELECT * FROM Payment WHERE payment_id=? AND booking_id=?", (payment_id, booking_id))
    updated_payment = cursor.fetchone()
    payment_dict = {
        "payment_id": updated_payment[0],
        "booking_id": updated_payment[1],
        "payment_type": updated_payment[2],
        "amount": updated_payment[3],
        "Payment_date": updated_payment[4]
    }

    return jsonify({"message": "Payment updated successfully", "payment": payment_dict}), 200


@app.route('/Payment/<int:payment_id>/<int:booking_id>', methods=['DELETE'])
def delete_Payment(payment_id, booking_id):
    cursor.execute("SELECT * FROM Payment WHERE payment_id = ? AND booking_id = ?", (payment_id, booking_id))
    payment = cursor.fetchone()
    if not payment:
        return jsonify({"error": "Payment not found"}), 404

    cursor.execute("DELETE FROM Payment WHERE payment_id = ? AND booking_id = ?", (payment_id, booking_id))
    conn.commit()
    return jsonify({"message": "Payment deleted successfully"}), 200




if __name__ == '__main__':
    app.run(debug=True, port=5001, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
